# Getting Started 3: Classic Machine Learning for Cybersecurity

**Open notebook on:** 
[![View filled on Github](https://img.shields.io/static/v1.svg?logo=github&label=Repo&message=View%20On%20Github&color=lightgrey)](https://github.com/clandolt)
[![Open filled In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/)   
**Author:** Christoph R. Landolt


In this tutorial, we will use a **basic machine learning algorithm** to gain insights into network traffic and detect potential cyber attacks. We’ll demonstrate:

- **Data preprocessing**
- **Feature extraction**
- **Training a classic ML model with scikit-learn**
- **Evaluating the model**